# Aprendizaje por refuerzo
# Importación de bibliotecas

In [7]:

from tqdm import tqdm
import numpy as np
import pandas as pd
import pickle

# Clase juego :gestiona el juego y el entrenamiento

In [8]:
class Game:
    def __init__(self, player1, player2):
        player1.symbol = 1
        player2.symbol = -1
        self.players = [player1, player2]
        self.board = Board()
        self.wins_over_time = [[0], [0]]
        self.explore_exploit_counts = [[], []]

    def selfplay(self, rounds=100):
        # Entrenamiento mediante self-play (juego contra sí mismos) durante 'rounds' partidas
        wins = [0, 0]
        for i in tqdm(range(1, rounds + 1)):  #
            self.board.reset()
            for player in self.players:
                player.reset()
            game_over = False
            while not game_over:  # Mientras la partida no termine
                for ix, player in enumerate(self.players):
                    # Realizar un movimiento (action) y registrar si fue exploración o explotación
                    action, explored = player.move(self.board, explore=True)
                    self.explore_exploit_counts[ix].append(1 if explored else 0)  # 1=exploró, 0=explotó
                    self.board.update(player.symbol, action)  # Actualizar el tablero con el movimiento
                    for player in self.players:
                        player.update(self.board)  # Actualizar el estado interno de cada agente
                    if self.board.is_game_over() is not None:  # Verificar si la partida terminó
                        game_over = True
                        break
            self.reward()
            for ix, player in enumerate(self.players):
                if self.board.is_game_over() == player.symbol:
                    wins[ix] += 1  # Incrementar victorias del jugador ganador
                self.wins_over_time[ix].append(wins[ix])  # Guardar victorias acumuladas
        return wins  # Devolver las victorias totales de cada jugador

    def reward(self):
        winner = self.board.is_game_over()
        if winner == 0:  # Empate
            for player in self.players:
                player.reward(0.5)  # Ambos reciben 0.5
        else:
            for player in self.players:
                if winner == player.symbol:
                    player.reward(1)
                else:
                    player.reward(0)


# Clase Agente: Representa a un agente que aprende a jugar usando aprendizaje por refuerzo

In [9]:
class Agent:
    def __init__(self, alpha=0.5, prob_exp=0.5):
        self.value_function = {}  # Diccionario para almacenar valores de estados (estado -> valor)
        self.alpha = alpha  # Tasa de aprendizaje
        self.positions = []  # Lista de estados visitados en una partida
        self.prob_exp = prob_exp  # Probabilidad de exploración (ε en ε-greedy)

    def reset(self):
        self.positions = []

    def move(self, board, explore=True):
        # Decidir el próximo movimiento del agente
        valid_moves = board.valid_moves()  # Obtener columnas válidas
        explored = False
        if explore and np.random.uniform(0, 1) < self.prob_exp:  # Exploración (ε-greedy)
            ix = np.random.choice(len(valid_moves))
            return valid_moves[ix], True  # Devuelve columna aleatoria y marca como exploración
        # Explotación: elegir el movimiento con mayor valor
        max_value = -1000
        best_col = valid_moves[0]
        for col in valid_moves:
            next_board = board.state.copy()
            board_copy = Board()
            board_copy.state = next_board
            board_copy.update(self.symbol, col)  # Simular el movimiento
            next_state = str(next_board.reshape(6*7))  # Convertir estado a cadena
            value = 0 if self.value_function.get(next_state) is None else self.value_function.get(next_state)
            if value >= max_value:
                max_value = value
                best_col = col
        return best_col, False  # Devuelve la mejor columna y marca como explotación

    def update(self, board):
        # Guardar el estado actual del tablero en la lista de posiciones
        self.positions.append(str(board.state.reshape(6*7)))

    def reward(self, reward):
        # Actualizar los valores de los estados al final de la partida
        for p in reversed(self.positions):  # Iterar en orden inverso
            if self.value_function.get(p) is None:
                self.value_function[p] = 0  # Inicializar valor en 0 si no existe
            # Actualizar valor usando aprendizaje por diferencias temporales (TD)
            self.value_function[p] += self.alpha * (reward - self.value_function[p])
            reward = self.value_function[p]  # Propagar el valor actualizado


# Clase Board: Representa el entorno donde se realizara el juego

In [10]:
# Clase Board: Representa el tablero de 4 en raya (6 filas x 7 columnas)
class Board:
    def __init__(self):
        self.rows = 6
        self.cols = 7
        self.state = np.zeros((self.rows, self.cols), dtype=int)  # Tablero inicial vacío

    def reset(self):
        self.state = np.zeros((self.rows, self.cols), dtype=int)

    def valid_moves(self):
        # Devolver columnas donde se puede jugar (no llenas)
        return [col for col in range(self.cols) if self.state[0, col] == 0]

    def update(self, symbol, col):
        # Colocar una ficha en la columna indicada (cae al fondo)
        for row in range(self.rows-1, -1, -1):
            if self.state[row, col] == 0:
                self.state[row, col] = symbol
                break

    def is_game_over(self):
        # Horizontal
        for row in range(self.rows):
            for col in range(self.cols-3):
                if (self.state[row, col] != 0 and
                    self.state[row, col] == self.state[row, col+1] == self.state[row, col+2] == self.state[row, col+3]):
                    return self.state[row, col]
        # Vertical
        for row in range(self.rows-3):
            for col in range(self.cols):
                if (self.state[row, col] != 0 and
                    self.state[row, col] == self.state[row+1, col] == self.state[row+2, col] == self.state[row+3, col]):
                    return self.state[row, col]
        # Diagonal positiva
        for row in range(self.rows-3):
            for col in range(self.cols-3):
                if (self.state[row, col] != 0 and
                    self.state[row, col] == self.state[row+1, col+1] == self.state[row+2, col+2] == self.state[row+3, col+3]):
                    return self.state[row, col]
        # Diagonal negativa
        for row in range(3, self.rows):
            for col in range(self.cols-3):
                if (self.state[row, col] != 0 and
                    self.state[row, col] == self.state[row-1, col+1] == self.state[row-2, col+2] == self.state[row-3, col+3]):
                    return self.state[row, col]
        # Empate (tablero lleno)
        if len(self.valid_moves()) == 0:
            return 0
        return None



# Ejecucion del Entrenamiento

In [11]:
# Ejecución del entrenamiento y visualización
agent1 = Agent(prob_exp=0.5)
agent2 = Agent()

game = Game(agent1, agent2)
game.selfplay(25000)

# Calcular proporción de exploración (promedio móvil para suavizar)
def moving_average(data, window_size=1000):
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')



100%|██████████| 25000/25000 [05:59<00:00, 69.49it/s]


In [13]:
# Guardar resultados en un DataFrame y mostrarlos
funcion_de_valor = sorted(agent1.value_function.items(), key=lambda kv: kv[1], reverse=True)
tabla = pd.DataFrame({'estado': [x[0] for x in funcion_de_valor], 'valor': [x[1] for x in funcion_de_valor]})
print(tabla)

# Guardar la función de valor en un archivo pickle
with open('agente_connect4.pickle', 'wb') as handle:
    pickle.dump(agent1.value_function, handle, protocol=pickle.HIGHEST_PROTOCOL)

                                                   estado  valor
0       [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...    1.0
1       [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...    1.0
2       [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...    1.0
3       [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...    1.0
4       [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...    1.0
...                                                   ...    ...
321827  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...    0.0
321828  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...    0.0
321829  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...    0.0
321830  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...    0.0
321831  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...    0.0

[321832 rows x 2 columns]
